In [3]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

import matplotlib.pyplot as plt


In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [5]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [6]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
TRAIN_SAMPLES = 100
TEST_SAMPLES = 50

In [10]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset1.data = dataset1.data[:TRAIN_SAMPLES]
dataset1.targets = dataset1.targets[:TRAIN_SAMPLES]


dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)
dataset2.data = dataset2.data[:TEST_SAMPLES]
dataset2.targets = dataset2.targets[:TEST_SAMPLES]


train_loader = torch.utils.data.DataLoader(dataset1)
test_loader = torch.utils.data.DataLoader(dataset2)

In [11]:
device = torch.device("cuda")

In [12]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters())

In [14]:
epochs = 14
gamma = 0.7


scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)    
    scheduler.step()

Train Epoch: 1 [0/100 (0%)]	Loss: 1.619173
Train Epoch: 1 [1/100 (1%)]	Loss: 0.000013
Train Epoch: 1 [2/100 (2%)]	Loss: 0.000809
Train Epoch: 1 [3/100 (3%)]	Loss: 0.001119
Train Epoch: 1 [4/100 (4%)]	Loss: 0.062622
Train Epoch: 1 [5/100 (5%)]	Loss: 0.502362
Train Epoch: 1 [6/100 (6%)]	Loss: 0.008939
Train Epoch: 1 [7/100 (7%)]	Loss: 0.559887
Train Epoch: 1 [8/100 (8%)]	Loss: 0.184912
Train Epoch: 1 [9/100 (9%)]	Loss: 0.012833
Train Epoch: 1 [10/100 (10%)]	Loss: 0.004528
Train Epoch: 1 [11/100 (11%)]	Loss: 0.090127
Train Epoch: 1 [12/100 (12%)]	Loss: 0.033102
Train Epoch: 1 [13/100 (13%)]	Loss: 0.025147
Train Epoch: 1 [14/100 (14%)]	Loss: 0.003424
Train Epoch: 1 [15/100 (15%)]	Loss: 0.148683
Train Epoch: 1 [16/100 (16%)]	Loss: 0.021773
Train Epoch: 1 [17/100 (17%)]	Loss: 0.003682
Train Epoch: 1 [18/100 (18%)]	Loss: 0.059360
Train Epoch: 1 [19/100 (19%)]	Loss: 0.345894
Train Epoch: 1 [20/100 (20%)]	Loss: 0.000505
Train Epoch: 1 [21/100 (21%)]	Loss: 0.000165
Train Epoch: 1 [22/100 (22%)]	

In [ ]:
n_samples_show = 6
count = 0
fig, axes = plt.subplots(nrows=1, ncols=n_samples_show, figsize=(10, 3))

model.eval()
for batch_idx, (data, target) in enumerate(test_loader):
    if count == n_samples_show:
        break
    output = model(data[0:1])
    if len(output.shape) == 1:
        output = output.reshape(1, *output.shape)

    pred = output.argmax(dim=1, keepdim=True)

    axes[count].imshow(data[0].numpy().squeeze(), cmap="gray")

    axes[count].set_xticks([])
    axes[count].set_yticks([])
    axes[count].set_title("Predicted {}".format(pred.item()))

    count += 1